<a href="https://colab.research.google.com/github/TommasoBuoso/spam_filter/blob/main/AI_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install sklearn
import numpy as np
import copy
import math
spambase = np.genfromtxt('/gdrive/My Drive/IA1/spambase/spambase.data', delimiter=',')
label = copy.deepcopy([i[-1] for i in spambase])
spambase = np.delete(spambase, np.s_[-4:], 1)

### TF-IDF COMPUTATION

In [3]:
N = len(spambase)

df = np.zeros(len(spambase[0]))
for i in spambase:
  df[i > 0] += 1 

w = copy.deepcopy(spambase)
for j in range(len(spambase)):
  for i in range(len(spambase[j])):
    w[j][i] = spambase[j][i] * np.log10(N/df[i])
print(w[0])

[0.         0.45412857 0.24758416 0.         0.13449946 0.
 0.         0.         0.         0.         0.         0.18971474
 0.         0.         0.         0.18210574 0.         0.83418479
 0.29732281 0.         0.26735887 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.24050181 0.         0.        ]


In [7]:
True in np.isnan(w)

False

In [8]:
w_angular = [i / np.sum(i ** 2) for i in w] #angular kernel if combined with linear
w_angular = np.nan_to_num(w_angular)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


# SVM

In [ ]:
from sklearn import svm, model_selection, metrics

X_train, X_test, y_train, y_test = model_selection.train_test_split(w, label, test_size = 0.33, random_state = 2)

## Linear

In [ ]:
clf = svm.SVC(kernel = "linear")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))

[[855  38]
 [ 79 547]]


In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.96      0.94       893
         1.0       0.94      0.87      0.90       626

    accuracy                           0.92      1519
   macro avg       0.93      0.92      0.92      1519
weighted avg       0.92      0.92      0.92      1519



In [ ]:
clf = svm.SVC(kernel = "linear")
np.mean(model_selection.cross_val_score(clf, w, label, cv = 10))

0.9095817221541074

In [9]:
#Model with angular informations
from sklearn import svm, model_selection, metrics

X_train, X_test, y_train, y_test = model_selection.train_test_split(w_angular, label, test_size = 0.33, random_state = 2)
clf = svm.SVC(kernel = "linear")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
np.mean(model_selection.cross_val_score(clf, w_angular, label, cv = 10))

0.9013246251061021

## Polynomial of degree 2

In [ ]:
clf = svm.SVC(kernel = "poly", degree=2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))

[[877  16]
 [315 311]]


In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.74      0.98      0.84       893
         1.0       0.95      0.50      0.65       626

    accuracy                           0.78      1519
   macro avg       0.84      0.74      0.75      1519
weighted avg       0.82      0.78      0.76      1519



In [ ]:
clf = svm.SVC(kernel = "poly", degree=2)
np.mean(model_selection.cross_val_score(clf, w, label, cv = 10))

0.8078756012449307

## RBF

In [ ]:
clf = svm.SVC(kernel = "rbf")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))

[[855  38]
 [ 98 528]]


In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.96      0.93       893
         1.0       0.93      0.84      0.89       626

    accuracy                           0.91      1519
   macro avg       0.92      0.90      0.91      1519
weighted avg       0.91      0.91      0.91      1519



In [ ]:
clf = svm.SVC(kernel = "rbf")
np.mean(model_selection.cross_val_score(clf, w, label, cv = 10))

0.9152324813731962

In [ ]:
#Exploring C parameter as explained in https://scikit-learn.org/stable/modules/svm.html#parameters-of-the-rbf-kernel
#res = []
#for i in range(1, 200):
#  clf = svm.SVC(kernel = "rbf", C = i) #gamma leaved to default, because we will perform cross validation
#  res.append((i, np.mean(model_selection.cross_val_score(clf, w, label, cv = 10))))

#res.sort(key=lambda tup: tup[1])
#print('Best score:', res[-1][1], ', with best parameter C:', res[-1][0])
#Best score: 0.9317504479864189 , with best parameter C: 170

Best score: 0.9317504479864189 , with best parameter C: 170


# NAIVE BAYES

In [ ]:
#Computation of alpha, mu and sigma^2
def naive_train(X_train, y_train):
  alpha = np.zeros(2)
  mu = np.zeros((2, len(X_train[0])))
  sigma2 = np.zeros((2, len(X_train[0])))

  for i in range(len(X_train)):
    alpha[ int(y_train[i]) ] += 1
    mu[ int(y_train[i]) ][ X_train[i] > 0 ] += 1

  alpha = alpha / N

  mu[0] = mu[0] / N
  mu[1] = mu[1] / N

  sigma2[0] = mu[0] - mu[0]**2
  sigma2[1] = mu[1] - mu[1]**2
  
  return alpha, mu, sigma2

In [ ]:
def guassian(test_row, mu, sigma2, k):
  res = 1

  for i in range(len(test_row)):
    res *= ( 2*math.pi*sigma2[k][i] ) ** (-1/2) * math.exp(-( 1 / (2*sigma2[k][i]) ) * (  int(test_row[i] > 0) - mu[k][i]) ** 2 )
    
  return res

def naive_bayes_clf(X_test, mu, sigma2, alpha):
  y_pred = []

  for row in X_test:
    res = ( guassian(row, mu, sigma2, 1) * alpha[1] ) / ( ( guassian(row, mu, sigma2, 1) * alpha[1] ) + ( guassian(row, mu, sigma2, 0) * alpha[0] ) )
    y_pred.append( int( res > 0.5 ) )

  return y_pred

In [ ]:
alpha, mu, sigma2 = naive_train(X_train, y_train)
y_pred = naive_bayes_clf(X_test, mu, sigma2, alpha)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))

[[693 200]
 [ 50 576]]


In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.78      0.85       893
         1.0       0.74      0.92      0.82       626

    accuracy                           0.84      1519
   macro avg       0.84      0.85      0.83      1519
weighted avg       0.85      0.84      0.84      1519



In [ ]:
metrics.accuracy_score(y_test, y_pred)

0.8354180381830152

In [ ]:
#Cross validation by hand
def cross_val(X, y, cv):
  res = []
  split_X = np.asarray( np.array_split(X, cv) )
  split_y = np.asarray( np.array_split(y, cv) )

  for i in range(cv):
    X_train =  np.concatenate( np.concatenate( (split_X[:i], split_X[i+1:])) )
    y_train =  np.concatenate( np.concatenate( (split_y[:i], split_y[i+1:])) )
    X_test = split_X[i]
    y_test = split_y[i]

    alpha, mu, sigma2 = naive_train(X_train, y_train)
    y_pred = naive_bayes_clf(X_test, mu, sigma2, alpha)

    res.append(metrics.accuracy_score(y_test, y_pred))
  
  return np.mean(res)

In [ ]:
print(cross_val(w, label, 10))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


0.732196548146751


In [ ]:
#Sklearn classifier to compare results
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(metrics.confusion_matrix(y_test, y_pred))

[[642 251]
 [ 28 598]]


In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.96      0.72      0.82       893
         1.0       0.70      0.96      0.81       626

    accuracy                           0.82      1519
   macro avg       0.83      0.84      0.82      1519
weighted avg       0.85      0.82      0.82      1519



In [ ]:
clf = GaussianNB()
np.mean(model_selection.cross_val_score(clf, w, label, cv = 10))

0.8150348957842122

# KNN

## OUR KNN

In [ ]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(5)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))

[[845  48]
 [ 79 547]]


In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.91      0.95      0.93       893
         1.0       0.92      0.87      0.90       626

    accuracy                           0.92      1519
   macro avg       0.92      0.91      0.91      1519
weighted avg       0.92      0.92      0.92      1519



In [ ]:
clf = neighbors.KNeighborsClassifier(5)
np.mean(model_selection.cross_val_score(clf, w, label, cv = 10))

0.8861062906724513

## KNN FROM INTERNET

In [ ]:
from math import sqrt

# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)
 
# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = []
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = []
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

# Make a classification prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
  neighbors = get_neighbors(train, test_row, num_neighbors)
  output_values = [row[-1] for row in neighbors]
  prediction = max(set(output_values), key=output_values.count)
  return prediction

def test_kNN(X_train, X_test, y_train, k):
  y_pred = []
  train = np.concatenate((np.asarray(X_train), np.asarray([y_train]).T), axis=1)
  for test_row in X_test:
    y_pred.append(predict_classification(train, test_row, k))
  
  return y_pred

In [ ]:
y_pred = test_kNN(X_train, X_test, y_train, 5)

print(metrics.confusion_matrix(y_test, y_pred))

[[837  49]
 [ 71 562]]


In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.94      0.93       886
         1.0       0.92      0.89      0.90       633

    accuracy                           0.92      1519
   macro avg       0.92      0.92      0.92      1519
weighted avg       0.92      0.92      0.92      1519

